In [1]:
import pandas as pd

df = pd.read_csv('BIT_2023_2차 (1).csv')
df.head(3)

,Unnamed: 0,Open,High,Low,Close,Volume,returns,volatility,price,quote_qty,...,upper_band_10,lower_band_10,roc_1,roc_2,rsi_7,rsi_9,rsi_14,UO_71014,UO_7911,UO_71012
0,2023-01-01 0:00,16537.5,16540.9,16504.0,16527.0,5381.399,-0.000114,0.002974,16524.52574,7264.129209,...,42885.07396,42180.51581,0.141586,0.093181,100.000000,100.000000,100.000000,0.000000,0.000000,0.000000
1,2023-01-01 1:00,16527.1,16554.3,16524.1,16550.4,3210.826,0.001416,0.002974,16537.21599,6819.889969,...,42885.07396,42180.51581,0.141586,0.093181,100.000000,100.000000,100.000000,40.577816,40.577816,40.577816
2,2023-01-01 2:00,16550.5,16557.1,16534.8,16542.4,2399.668,-0.000483,0.002974,16545.81814,6030.420093,...,42885.07396,42180.51581,-0.048337,0.093181,74.522293,74.522293,74.522293,45.684084,45.684084,45.684084


In [2]:
df = df.rename(columns={'Unnamed: 0': 'time'})
df = df.set_index('time')
df.index=pd.to_datetime(df.index)

## Only with random_state

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error

## 'volatility'와의 상관 계수 값 > 0 column

In [ ]:
# 상관계수
correlations = df.corr()['volatility']

# 'volatility'와의 상관 계수 값 > 0 컬럼
selected_columns = correlations[correlations > 0].index

X = df[selected_columns].drop(columns=['volatility'])
y = df['volatility']

# 훈련 데이터 / 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 생성 + 학습
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# 테스트 데이터 -> 예측 수행
y_pred = model.predict(X_test)

# MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)
print('MAPE:', mape)

## Optimal Hyper-Parameter_Randomized Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# 랜덤 포레스트 모델 생성
model = RandomForestRegressor()

# 탐색할 하이퍼파라미터들 -> 딕셔너리 형태로 지정
param_grid = {
    'n_estimators': np.arange(100, 400, 100),
    'max_depth': [None, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': np.arange(2, 7, 2),
    'min_samples_leaf': np.arange(1, 4),
    'max_leaf_nodes': [None, 10, 20]
}

# RandomizedSearchCV를 사용하여 최적의 조합 탐색
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=5)
random_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 조합 출력
print("Best Parameters:", random_search.best_params_)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error

# 최적의 하이퍼파라미터로 랜덤 포레스트 모델 생성
model = RandomForestRegressor(n_estimators=100, max_depth=None, max_features='auto',
                              min_samples_split=2, min_samples_leaf=2, max_leaf_nodes=None)

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측값
y_pred = model.predict(X_test)

# MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)

# MAPEㄴ
print("MAPE:", mape)